# Gated Recurrent Unit

In [1]:
from d2l.tensorflow import data, losses, optimizers, models

import tensorflow as tf
import numpy as np

In [2]:
data_iter, vocab = data.load_seq_data('timemachine.txt', 128, 1)

In [3]:
vocab_size = len(vocab)
corpus_indices = data_iter.corpus
idx_to_char, char_to_idx = vocab.idx_to_token, vocab.token_to_idx
num_inputs, num_hiddens, num_outputs = vocab_size, 512, vocab_size

def get_params():
    def _one(shape):
        return tf.random.normal(stddev=0.01, shape=shape)

    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                tf.zeros(num_hiddens))

    W_xz, W_hz, b_z = _three()  # Update gate parameter
    W_xr, W_hr, b_r = _three()  # Reset gate parameter
    W_xh, W_hh, b_h = _three()  # Candidate hidden state parameter
    # Output layer parameters
    W_hq = _one((num_hiddens, num_outputs))
    b_q = tf.zeros(num_outputs)
    # Create gradient
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    return [tf.Variable(param) for param in params]

## GRU Model

In [4]:
def init_gru_state(batch_size, num_hiddens):
    state = [tf.zeros(shape=(batch_size, num_hiddens)), ]
    return [tf.Variable(st) for st in state]

In [5]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = tf.sigmoid(tf.matmul(X, W_xz) + tf.matmul(H, W_hz) + b_z)
        R = tf.sigmoid(tf.matmul(X, W_xr) + tf.matmul(H, W_hr) + b_r)
        H_tilda = tf.tanh(tf.matmul(X, W_xh) + R * tf.matmul(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = tf.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

## Train the Model

In [6]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['traveller', 'time traveller']
loss = losses.softmax_cross_entropy
optimizer = optimizers.sgd

In [7]:
# models.train_and_predict_rnn(
#     gru, loss, optimizer, get_params, init_gru_state, num_hiddens,
#     vocab_size, corpus_indices, idx_to_char,
#     char_to_idx, False, num_epochs, num_steps, lr,
#     clipping_theta, batch_size, pred_period, pred_len,
#     prefixes
# )